In [ ]:
import numpy as np
import json
import itertools
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
from symd import prepare_input

# Loading groups

In [ ]:
prepare_input(15, 2, 24, "wp-15")

In [ ]:
# check them all
for i in range(1, 18):
    prepare_input(i, 2, 16, "test")
for i in range(1, 231):
    prepare_input(i, 3, 16, "test")

## Making Pictures

In [ ]:
import matplotlib.patches as patches
from symd.groups import *


def CubicEaseInOut(p):
    if p < 0.5:
        return 4 * p * p * p
    else:
        f = (2 * p) - 2
        return 0.5 * f * f * f + 1


def plot_group(
    axis,
    x,
    basis,
    g,
    title,
    show_basis=False,
    colors="black",
    n_images=2,
    background_color="#f5f4e9",
    alpha=0.5,
):
    basis = (g[0] @ basis.flatten()).reshape(basis.shape)
    if show_basis:
        axis.plot([0, basis[0, 0]], [0, basis[1, 0]], "-", color="black")
        axis.plot([0, basis[0, 1]], [0, basis[1, 1]], "-", color="black")
    n = list(range(-n_images, n_images + 1))
    # reduce x size to match group
    # so we have approximately same points no matter which group
    npoints = x.shape[0] // len(g[1])
    npoints = x.shape[0] if npoints == 0 else npoints
    x = x[:npoints, :]
    points = []
    if type(colors) is str:
        colors = [colors] * x.shape[0]
    for i, ns in enumerate(itertools.product(n, repeat=2)):
        for w in g[1]:
            xw = np.mod((w @ x.T).T, 1)
            xc = (xw[:, :2] + ns) @ basis.T
            if colors is None:
                c = np.array(["white"] * xc.shape[0])
                c[xc[:, 1] > 0] = "#005bbb"
                c[xc[:, 1] < 0] = "#ffd500"
                axis.scatter(
                    xc[:, 0], xc[:, 1], c=c, marker=".", alpha=alpha, edgecolors="none"
                )
            else:
                axis.scatter(
                    xc[:, 0],
                    xc[:, 1],
                    c=colors[: xc.shape[0]],
                    marker=".",
                    alpha=alpha,
                    zorder=100,
                    edgecolors="none",
                )
            points.append(xc[:, :2])
    x = np.vstack(points)
    # sns.kdeplot(x=x[:,0], y=x[:,1], shade=True, bw_adjust=0.2, cmap='Reds')
    if title:
        axis.set_title(title, fontsize=18, color="#333333", fontname="monospace")
    axis.set_aspect("equal")
    axis.axis("off")
    axis.set_facecolor(background_color)
    plt.gcf()


def generate_points(x, basis, g):
    basis = (g[0] @ basis.flatten()).reshape(basis.shape)
    n = [0, -1, 1, -2, 2]
    # reduce x size to match group
    npoints = x.shape[0] // len(g[1])
    x = x[:npoints, :]
    points = []
    for i, ns in enumerate(itertools.product(n, repeat=2)):
        for w in g[1]:
            xw = np.mod((w @ x.T).T, 1)
            xc = (xw[:, :2] + ns) @ basis
            points.append(xc[:, :2])
    x = np.vstack(points)
    return x


np.random.seed(0)
npoints = 96
x = np.hstack((np.random.uniform(size=(npoints, 2)) - 0.5, np.ones((npoints, 1))))
basis = np.array([[1, 0], [0, 1]]) + np.random.uniform(size=(2, 2)) * 0.5
base_colors = [
    "f94144",
    "f3722c",
    "f8961e",
    "f9844a",
    "f9c74f",
    "90be6d",
    "43aa8b",
    "4d908e",
    "577590",
    "277da1",
]
colors = ["#" + base_colors[i % len(base_colors)] for i in range(x.shape[0])]
background_color = "#f5f4e9"
titles = [
    "p1",
    "p2",
    "pm",
    "pg",
    "cm",
    "pmm",
    "pmg",
    "pgg",
    "cmm",
    "p4",
    "p4m",
    "p4g",
    "p3",
    "p3m1",
    "p31m",
    "p6",
    "p6m",
]
mosaic = [
    "A" * 5,
    "B" * 5,
    "C" * 5,
    "D" * 5,
    "E" * 5,
    "F" * 5,
    "\n",
    "G" * 5,
    "H" * 5,
    "I" * 5,
    "J" * 5,
    "K" * 5,
    "L" * 5,
    "\n",
    "M" * 6,
    "N" * 6,
    "O" * 6,
    "P" * 6,
    "Q" * 6,
    "\n",
]
names = "ABCDEFGHIJKLMNOPQ"
fig = plt.figure(constrained_layout=False, figsize=(18, 12))
axes = fig.subplot_mosaic("".join(mosaic))
# clear out
for n in names:
    a = axes[n]
    a.axis("off")
    a.set_facecolor(background_color)
for i in range(1, 18):
    group = load_group(i, 2)
    fig.patch.set_facecolor(background_color)
    axis = axes[names[i - 1]]
    projector = projectors2d[group["lattice"]]
    members = [str2mat(i) for i in group["genpos"]]
    plot_group(
        axis,
        x,
        basis,
        (projector, members),
        title=titles[i - 1],
        show_basis=False,
        n_images=1,
        colors="#333",
        background_color=background_color,
        alpha=0.6,
    )
    for j, sp in enumerate(group["specpos"]):
        members = [str2mat(i) for i in sp["sites"]]
        sub_colors = ["#" + base_colors[j]] * x.shape[0]
        plot_group(
            axis,
            x[:3],
            basis,
            (projector, members),
            title=None,
            show_basis=False,
            n_images=1,
            colors=sub_colors,
            background_color=background_color,
            alpha=0.6,
        )
plt.tight_layout()
plt.savefig(f"wallpaper.jpg", dpi=300)

In [ ]:
# make a movie
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

data = []
titles = []
for k, v in wallpaper.items():
    data.append(generate_points(x, basis, v))
    titles.append(k)

# make it loop
data.append(data[0])
titles.append(titles[0])

scale = 2
duration = (len(data) - 1) * scale
dpi = 90
fig, ax = plt.subplots(figsize=(800 / dpi, 800 / dpi), dpi=dpi)
points = ax.plot(
    data[0][:, 0],
    data[1][:, 1],
    color="#333333",
    marker=".",
    markeredgewidth=0.0,
    linestyle="None",
    alpha=0.2,
)[0]
ax.set_facecolor("#f5f4e9")
fig.patch.set_facecolor("#f5f4e9")
title = ax.set_title("", fontsize=32, color="#333333", fontname="monospace")
ax.set_aspect(1.0 / ax.get_data_ratio(), adjustable="box")
ax.axis("off")
# ax.set_xlim(-3,3)
# ax.set_ylim(-3,3)
# plt.tight_layout()
def make_frame(t):
    t /= scale
    s = t % 1
    if s < 0.25:
        s = CubicEaseInOut(s * 4)
    else:
        s = 1
    i = int(t) + 1
    p = data[i] * s + (1 - s) * data[i - 1]
    title.set_text(titles[i])
    points.set_data(p[:, 0], p[:, 1])
    ax.set_facecolor("#f5f4e9")
    return mplfig_to_npimage(fig)


animation = VideoClip(make_frame, duration=duration)
animation.write_gif("matplotlib.gif", fps=15)

## Generating Groups

**Should not need to execute!!**

In [ ]:
import time
import requests
from bs4 import BeautifulSoup as BS


def get_html(group_no, dim=3):
    if dim == 3:
        url = f"https://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o{group_no:03}/"
    elif dim == 2:
        url = f"https://it.iucr.org/Ac/ch2o2v0001/sgtable2o2o{group_no:03}/"
    result = requests.get(url)
    return result.content


def combine(s, t):
    """
    Combine string of coords. Example:
    combine("x, y, z - y", "0, 0, 1/2")
    will return "x, y, z - y + 1/2"
    """
    s = s.split(",")
    t = t.split(",")
    for i in range(len(s)):
        s[i] = s[i] + " + " + t[i]
    return ",".join(s)


def parse_trans_str(text):
    """
    Split trans string into individual pieces
    """
    text = text.replace("+", "")
    r = []
    for t in text.split("("):
        r.append(t.replace(")", ""))
    # filter out empties
    return [ri for ri in r if ri]


def parse_group(html):
    b = BS(html)
    gen_table = b.select_one("div > table.genpos")
    # sorry
    trans_str = []
    try:
        # find the first row after header
        index = 0
        for tr in gen_table.find_all("tr"):
            if "Coordinates" in tr.text:
                continue
            if len(tr.text) < 5:
                continue
            break
        trans_str = "".join(tr.find_all("td")[1].text.split())
        trans_str = parse_trans_str(trans_str)
    except IndexError:
        pass
    intable = False
    sites = []
    for t in gen_table.find_all("table"):
        for row in t.find_all("tr"):
            data = row.find_all("td")
            if intable:
                for d in data:
                    if len(d.text.split(")")) == 1:
                        intable = False
                        break
                    text = "".join(d.text.split(")")[-1].split())
                    if trans_str:
                        for tr in trans_str:
                            sites.append(combine(text, tr))
                    else:
                        sites.append(text)
            else:
                try:
                    multi = int(data[0].text)
                    letter = data[1].text
                    sym = data[2].text
                    sites = []
                    intable = True
                except:
                    pass
                    sites = []
    spec_table = b.select_one("div > table.specpos")
    specs = []
    if spec_table:
        # not sure why we cannot use tr.specpos
        intable = False
        wsites = None  # will be set - can't figure out why we're getting repeats
        for w in spec_table.find_all("tr"):
            info_table = w.select_one("table")
            if info_table:
                # save old one
                if wsites is not None:
                    # convert to matrices
                    wsites = list(wsites)
                    assert len(wsites) == mpc, (
                        "Only had sites " + str(wsites) + " but needed " + str(mpc)
                    )
                    specs.append({"name": name, "size": mpc, "sites": wsites})
                mpc, name, *_ = info_table.text.split()
                mpc = int(mpc)
                intable = True
                wsites = set()
            if intable:
                for td in w.find_all("td"):
                    if "class" in td.attrs and "specposcoords" in td.attrs["class"]:
                        s = "".join(td.text.split())
                        if trans_str:
                            for tr in trans_str:
                                c = combine(s, tr)
                                wsites.add(c)
                        else:
                            wsites.add(s)
        if wsites is not None:
            # convert to matrices
            wsites = list(wsites)
            assert len(wsites) == mpc, (
                "Only had sites " + str(wsites) + " but needed " + str(mpc)
            )
            specs.append({"name": name, "size": mpc, "sites": wsites})
    asymm = b.select("td.asymmetricunit")[1].text
    asymm = "".join(asymm.split())
    p = b.select_one("tr.sgheader").select("td.sgheader")[-1].text.strip()
    data = {"lattice": p, "genpos": sites, "asymm_unit": asymm, "specpos": specs}
    return data


def get_group(number, dim=3):
    return parse_group(get_html(number, dim=dim))

In [ ]:
all_groups = {}
for g in range(1, 18):
    time.sleep(1)
    group = get_group(g, 2)
    all_groups[g] = group
with open("2dgroups.json", "w") as f:
    json.dump(all_groups, f, indent=True)

In [ ]:
all_groups = {}
for g in range(1, 231):
    time.sleep(1)
    group = get_group(g, 3)
    all_groups[g] = group
with open("3dgroups.json", "w") as f:
    json.dump(all_groups, f, indent=True)